In [1]:
import sys
sys.path.append('../')

import tquant as tq

from datetime import date
import tensorflow as tf
# import pandas as pd
# import numpy as np
from datastore import *

# Settings

In [2]:
tq.Settings.evaluation_date = date(2024, 4, 30)

calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

# Market

In [3]:
rates = ir_eur_curve_estr['quote'].values
times = ir_eur_curve_estr['daycount'].values/365

rates_6m = ir_eur_curve_6m['quote'].values
times_6m = ir_eur_curve_6m['daycount'].values/365

In [4]:
estr_curve = tq.RateCurve(times, rates)
eur6m_curve = tq.RateCurve(times_6m, rates_6m)

In [5]:
# instance of the hull-white model for 1M curve simulation
mean_rev = 0.02
sigma = 0.045
hw = tq.HullWhiteProcess(mean_rev, sigma, market_curve=estr_curve)
hw_kernel = tq.GaussianRateKernel1D(hw)

# simulate short rate and discount curve 
paths = int(1e6) 
r_tensor, zb_tensor, grid = hw_kernel.rate_tensor(paths,
                                                  last_grid_time=1,
                                                  time_steps=2) # evaluation_date + 1y
# Reshape the tensor and create a discount curve object
reshaped_tensor = tf.reshape(zb_tensor, shape=(zb_tensor.shape[1], paths, 1))
rate_curve_tensor = tq.DiscountCurveSimple(times, reshaped_tensor)

# Coupons

## Fixed coupon 

In [6]:
d1 = calendar.advance(evaluation_date,
                         6,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing)
nominal = 1e6
fixed_coupon = tq.FixedCoupon(d1,
                              nominal,
                              accrual_start_date= evaluation_date,
                              accrual_end_date= d1,
                              ref_period_start= evaluation_date,
                              ref_period_end= d1,
                              r= 0.02,
                              daycounter = daycounter)

### prezzo di un singolo coupon

In [7]:
fixed_coupon_pricer = tq.FixedCouponDiscounting(fixed_coupon)
# price single coupon
fixed_coupon_pricer.price(estr_curve, evaluation_date)

<tf.Tensor: shape=(), dtype=float64, numpy=9978.535430352356>

### prezzo dello stesso coupon con 1M curve differenti

In [8]:
# price with tensor curves
fixed_coupon_pricer.price(rate_curve_tensor, evaluation_date)

<tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
array([[ 9975.94638503],
       [ 9872.76538324],
       [ 9860.31397776],
       ...,
       [10123.87152874],
       [10009.64972096],
       [ 9829.03652723]])>

### prezzo dello stesso coupon con 1M curve differenti a 50 date

In [9]:
# price with tensor curves for 50 dates
date_list = [evaluation_date] *50
[fixed_coupon_pricer.price(rate_curve_tensor, d) for d in date_list]

[<tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9975.94638503],
        [ 9872.76538324],
        [ 9860.31397776],
        ...,
        [10123.87152874],
        [10009.64972096],
        [ 9829.03652723]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9975.94638503],
        [ 9872.76538324],
        [ 9860.31397776],
        ...,
        [10123.87152874],
        [10009.64972096],
        [ 9829.03652723]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9975.94638503],
        [ 9872.76538324],
        [ 9860.31397776],
        ...,
        [10123.87152874],
        [10009.64972096],
        [ 9829.03652723]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9975.94638503],
        [ 9872.76538324],
        [ 9860.31397776],
        ...,
        [10123.87152874],
        [10009.64972096],
        [ 9829.03652723]])>,
 <tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
 array([[ 9975.94638503],
        [ 9

### AD sensitivity

In [10]:
pv, tape = fixed_coupon_pricer.price_aad(estr_curve, evaluation_date)

In [11]:
sensitivities = tape.gradient(pv, [estr_curve.rates])

In [12]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0]]).T
sensy.columns = ['time', 'sensitivity']
sensy[sensy['time'] > 0.3].head(10)

,time,sensitivity
9,0.334247,NaN
10,0.419178,NaN
11,0.501370,NaN
12,0.583562,NaN
13,0.668493,NaN
14,0.753425,NaN
15,0.832877,NaN
16,0.917808,NaN
17,1.000000,NaN
18,1.082192,NaN


## Floating coupon

In [13]:
d1 = calendar.advance(evaluation_date,
                         6,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing)

index = tq.IborIndex('EUR6M',
             calendar,
             6,
             tq.TimeUnit.Months)
gearing = 1
spread = 0
nominal = 1e6

floating_coupon = tq.FloatingCoupon(d1,
                                    nominal,
                                    evaluation_date,
                                    d1,
                                    index,
                                    gearing,
                                    spread,
                                    evaluation_date,
                                    d1,
                                    daycounter
                                    )

In [14]:
float_pricer = tq.FloatingCouponDiscounting(floating_coupon, False)

In [15]:
npv, tape = float_pricer.price_aad(estr_curve, eur6m_curve, tq.Settings.evaluation_date)
sensitivities = tape.gradient(npv, [estr_curve.rates, eur6m_curve.rates])

In [16]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0], sensitivities[1]]).T
sensy.columns = ['time', 'sensy_estr', 'sensy_eur6m']
sensy[sensy['time'] > 0.2].head(10)

,time,sensy_estr,sensy_eur6m
8,0.249315,NaN,NaN
9,0.334247,NaN,NaN
10,0.419178,NaN,NaN
11,0.50137,NaN,NaN
12,0.583562,NaN,NaN
13,0.668493,NaN,NaN
14,0.753425,NaN,NaN
15,0.832877,NaN,NaN
16,0.917808,NaN,NaN
17,1.0,NaN,NaN


# Legs

## Fixed Leg

In [18]:
fixed_schedule = [tq.TARGET().advance(evaluation_date,
                         t,
                         tq.TimeUnit.Months,
                         tq.BusinessDayConvention.ModifiedFollowing) for t in [0,6,12,18,24]]
notionals = [1e6] * (len(fixed_schedule) - 1)
coupon_rates = [0.03] * (len(fixed_schedule) - 1)

fixed_leg = tq.FixedRateLeg(fixed_schedule,
                            notionals,
                            coupon_rates,
                            daycounter= daycounter,
                            compounding=tq.CompoundingType.Simple,
                            frequency= tq.Frequency.Annual)

In [19]:
fixed_leg_pricer = tq.FixedLegDiscounting(fixed_leg)

In [20]:
fixed_leg_pricer.price(estr_curve, evaluation_date, tq.FixedCouponDiscounting)

<tf.Tensor: shape=(), dtype=float64, numpy=58332.03933251082>

In [21]:
fixed_leg_pricer.price(estr_curve, evaluation_date, tq.FixedCouponDiscounting)

<tf.Tensor: shape=(), dtype=float64, numpy=58332.03933251082>

In [22]:
npv, tape = fixed_leg_pricer.price_aad(estr_curve, evaluation_date, tq.FixedCouponDiscounting)
sensitivities = tape.gradient(npv, [estr_curve.rates, eur6m_curve.rates])

In [23]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0]]).T
sensy.columns = ['time', 'sensitivity']
sensy[sensy['time'] > 0.4].head(20)

,time,sensitivity
10,0.419178,NaN
11,0.501370,NaN
12,0.583562,NaN
13,0.668493,NaN
14,0.753425,NaN
15,0.832877,NaN
16,0.917808,NaN
17,1.000000,NaN
18,1.082192,NaN
19,1.167123,NaN


## Floating Leg

In [24]:
spreads = [0] * (len(fixed_schedule) - 1)
gearing = [1] * (len(fixed_schedule) - 1)
floating_schedule = fixed_schedule
index = tq.IborIndex('EUR6M',
             calendar,
             6,
             tq.TimeUnit.Months)

floating_leg = tq.FloatingRateLeg(floating_schedule,
                               notionals,
                               gearing,
                               spreads,
                               index,
                               daycounter)

In [25]:
floating_leg_pricer = tq.FloatingLegDiscounting(floating_leg)

In [26]:
floating_leg_pricer.price(estr_curve, eur6m_curve, evaluation_date, tq.FloatingCouponDiscounting)

<tf.Tensor: shape=(), dtype=float64, numpy=63830.13448251583>

In [27]:
npv, tape =floating_leg_pricer.price_aad(estr_curve, eur6m_curve, evaluation_date, tq.FloatingCouponDiscounting)
sensitivities = tape.gradient(npv, [estr_curve.rates, eur6m_curve.rates])

In [28]:
sensy = pd.DataFrame([estr_curve.pillars.tolist(), sensitivities[0], sensitivities[1]]).T
sensy.columns = ['time', 'sensy_estr', 'sensy_eur6m']
sensy[sensy['time'] > 0.2].head(10)

,time,sensy_estr,sensy_eur6m
8,0.249315,NaN,NaN
9,0.334247,NaN,NaN
10,0.419178,NaN,NaN
11,0.50137,NaN,NaN
12,0.583562,NaN,NaN
13,0.668493,NaN,NaN
14,0.753425,NaN,NaN
15,0.832877,NaN,NaN
16,0.917808,NaN,NaN
17,1.0,NaN,NaN
